# TalkingData AdTracking Fraud Detection Challenge

### Description

Fraud risk is everywhere, but for companies that advertise online, click fraud can happen at an overwhelming volume, resulting in misleading click data and wasted money. Ad channels can drive up costs by simply clicking on the ad at a large scale. With over 1 billion smart mobile devices in active use every month, China is the largest
mobile market in the world and therefore suffers from huge volumes of fradulent traffic.

TalkingData, China’s largest independent big data service platform, covers over 70% of active mobile devices nationwide. They handle 3 billion clicks per day, of which 90% are potentially fraudulent. Their current approach to prevent click fraud for app developers is to measure the journey of a user’s click across their portfolio, and flag IP addresses who produce lots of clicks, but never end up installing apps. With this information, they've built an IP blacklist and device blacklist.

While successful, they want to always be one step ahead of fraudsters and have turned to the Kaggle community for help in further developing their solution. In their 2nd competition with Kaggle, you’re challenged to build an algorithm that predicts whether a user will download an app after clicking a mobile app ad. To support your modeling, they have provided a generous dataset covering approximately 200 million clicks over 4 days!

### Evaluation
Submissions are evaluated on <b>area under the ROC curve </b> between the predicted probability and the observed target.

### Data fields
Each row of the training data contains a click record, with the following features.

<b>ip:</b> ip address of click.<br>
<b>app:</b> app id for marketing.<br>
<b>device:</b> device type id of user mobile phone (e.g., iphone 6 plus, iphone 7, huawei mate 7, etc.)<br>
<b>os:</b> os version id of user mobile phone<br>
<b>channel:</b> channel id of mobile ad publisher<br>
<b>click_time:</b> timestamp of click (UTC)<br>
<b>attributed_time:</b> if user download the app for after clicking an ad, this is the time of the app download<br>
<b>is_attributed:</b> the target that is to be predicted, indicating the app was downloaded<br>

Note that ip, app, device, os, and channel are encoded.

The test data is similar, with the following differences:

<b>click_id:</b> reference for making predictions<br>
<b>is_attributed:</b> not included

## Import Packages

In [1]:
import pandas as pd
import time
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score, roc_curve, auc, classification_report
from sklearn.model_selection import GridSearchCV

In [2]:
np.random.seed(42)

In [3]:
sub_file_name = '03-sub_lgb_balanced99.csv'

In [4]:
exp_path = '../../experiments/exp_3/'

## Import Data

In [5]:
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

print('loading train data...')

chunk = 40000000
#chunk = 87000000
start = 144903891
#start = 184903891-chunk

train_df = pd.read_csv('../../data/train.csv',  skiprows=range(1, start),  nrows=chunk, dtype=dtypes, 
                       usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])

print('loading test data...')
test_df = pd.read_csv('../../data/test.csv', dtype=dtypes, 
                      usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])

len_train = len(train_df)

# Join the datasets to apply the transformations only one time
train_df=train_df.append(test_df)

train_df.shape, test_df.shape

loading train data...
loading test data...


((58790469, 8), (18790469, 7))

In [6]:
print('Train Data:')
display(train_df.head())
display(train_df.tail())

print('Test Data:')
display(test_df.head())
display(test_df.tail())

Train Data:


,ip,app,device,os,channel,click_time,is_attributed,click_id
0,33924,15,1,19,111,2017-11-09 04:03:08,0.0,NaN
1,37383,3,1,13,280,2017-11-09 04:03:08,0.0,NaN
2,122294,15,1,10,245,2017-11-09 04:03:08,0.0,NaN
3,73258,9,1,25,145,2017-11-09 04:03:08,0.0,NaN
4,73347,15,1,13,430,2017-11-09 04:03:08,0.0,NaN


,ip,app,device,os,channel,click_time,is_attributed,click_id
18790464,99442,9,1,13,127,2017-11-10 15:00:00,NaN,18790464.0
18790465,88046,23,1,37,153,2017-11-10 15:00:00,NaN,18790465.0
18790466,81398,18,1,17,265,2017-11-10 15:00:00,NaN,18790467.0
18790467,123236,27,1,13,122,2017-11-10 15:00:00,NaN,18790466.0
18790468,73516,12,2,27,265,2017-11-10 15:00:00,NaN,18790468.0


Test Data:


,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


,click_id,ip,app,device,os,channel,click_time
18790464,18790464,99442,9,1,13,127,2017-11-10 15:00:00
18790465,18790465,88046,23,1,37,153,2017-11-10 15:00:00
18790466,18790467,81398,18,1,17,265,2017-11-10 15:00:00
18790467,18790466,123236,27,1,13,122,2017-11-10 15:00:00
18790468,18790468,73516,12,2,27,265,2017-11-10 15:00:00


In [7]:
del test_df
gc.collect()

20

## Feature Engineering

#### Extracting time information from click_time feature

In [8]:
train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')
train_df['minute'] = pd.to_datetime(train_df.click_time).dt.minute.astype('uint8')
train_df['second'] = pd.to_datetime(train_df.click_time).dt.second.astype('uint8')

gc.collect()

10

####  Group-By-Aggregation

In [9]:
# Define all the groupby transformations
GROUPBY_AGGREGATIONS = [
    # V1 - GroupBy Features #
    # Variance in day, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'day', 'agg': 'var'},
    # Variance in hour, for ip-app-os
    {'groupby': ['ip', 'app', 'os'], 'select': 'hour', 'agg': 'var'},
    # Variance in hour, for ip-day-channel
    {'groupby': ['ip','day','channel'], 'select': 'hour', 'agg': 'var'},
    # Count, for ip-day-hour
    {'groupby': ['ip','day','hour'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app
    {'groupby': ['ip', 'app'], 'select': 'channel', 'agg': 'count'},        
    # Count, for ip-app-os
    {'groupby': ['ip', 'app', 'os'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app-day-hour
    {'groupby': ['ip','app','day','hour'], 'select': 'channel', 'agg': 'count'},
    # Mean hour, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'hour', 'agg': 'mean'}, 
    # V2 - GroupBy Features #
    # Average clicks on app by distinct users; is it an app they return to?
    {'groupby': ['app'], 
     'select': 'ip', 
     'agg': lambda x: float(len(x)) / len(x.unique()), 
     'agg_name': 'AvgViewPerDistinct'
    },
    # How popular is the app or channel?
    {'groupby': ['app'], 'select': 'channel', 'agg': 'count'},
    {'groupby': ['channel'], 'select': 'app', 'agg': 'count'},
#     # V3 - GroupBy Features                                              #
#     # https://www.kaggle.com/bk0000/non-blending-lightgbm-model-lb-0-977 #
    {'groupby': ['ip'], 'select': 'channel', 'agg': 'nunique'}, 
    {'groupby': ['ip'], 'select': 'app', 'agg': 'nunique'}, 
    {'groupby': ['ip','day'], 'select': 'hour', 'agg': 'nunique'}, 
    {'groupby': ['ip','app'], 'select': 'os', 'agg': 'nunique'}, 
#     {'groupby': ['ip'], 'select': 'device', 'agg': 'nunique'}, 
#     {'groupby': ['app'], 'select': 'channel', 'agg': 'nunique'}, 
#     {'groupby': ['ip', 'device', 'os'], 'select': 'app', 'agg': 'nunique'}, 
#     {'groupby': ['ip','device','os'], 'select': 'app', 'agg': 'cumcount'}, 
#     {'groupby': ['ip'], 'select': 'app', 'agg': 'cumcount'}, 
#     {'groupby': ['ip'], 'select': 'os', 'agg': 'cumcount'}, 
#     {'groupby': ['ip','day','channel'], 'select': 'hour', 'agg': 'var'}    
]

# Apply all the groupby transformations
for spec in GROUPBY_AGGREGATIONS:
    # Name of the aggregation we're applying
    agg_name = spec['agg_name'] if 'agg_name' in spec else spec['agg']
    # Name of new feature
    new_feature = '{}_{}_{}'.format('_'.join(spec['groupby']), agg_name, spec['select'])
    # Info
    print("Grouping by {}, and aggregating {} with {}".format(
        spec['groupby'], spec['select'], agg_name))
    # Unique list of features to select
    all_features = list(set(spec['groupby'] + [spec['select']]))
    # Perform the groupby
    gp = train_df[all_features]. \
        groupby(spec['groupby'])[spec['select']]. \
        agg(spec['agg']). \
        reset_index(). \
        rename(index=str, columns={spec['select']: new_feature})
    # Merge back to X_total
    if 'cumcount' == spec['agg']:
        train_df[new_feature] = gp[0].values
    else:
        train_df = train_df.merge(gp, on=spec['groupby'], how='left')
        
    # Clear memory
    del gp
    gc.collect()

train_df.head()

Grouping by ['ip', 'app', 'channel'], and aggregating day with var
Grouping by ['ip', 'app', 'os'], and aggregating hour with var
Grouping by ['ip', 'day', 'channel'], and aggregating hour with var
Grouping by ['ip', 'day', 'hour'], and aggregating channel with count
Grouping by ['ip', 'app'], and aggregating channel with count
Grouping by ['ip', 'app', 'os'], and aggregating channel with count
Grouping by ['ip', 'app', 'day', 'hour'], and aggregating channel with count
Grouping by ['ip', 'app', 'channel'], and aggregating hour with mean
Grouping by ['app'], and aggregating ip with AvgViewPerDistinct
Grouping by ['app'], and aggregating channel with count
Grouping by ['channel'], and aggregating app with count
Grouping by ['ip'], and aggregating channel with nunique
Grouping by ['ip'], and aggregating app with nunique
Grouping by ['ip', 'day'], and aggregating hour with nunique
Grouping by ['ip', 'app'], and aggregating os with nunique


,ip,app,device,os,channel,click_time,is_attributed,click_id,hour,day,...,ip_app_os_count_channel,ip_app_day_hour_count_channel,ip_app_channel_mean_hour,app_AvgViewPerDistinct_ip,app_count_channel,channel_count_app,ip_nunique_channel,ip_nunique_app,ip_day_nunique_hour,ip_app_nunique_os
0,33924,15,1,19,111,2017-11-09 04:03:08,0.0,NaN,4,9,...,58,17,9.166667,41.046478,3535169,187255,95,45,12,34
1,37383,3,1,13,280,2017-11-09 04:03:08,0.0,NaN,4,9,...,77,57,5.215686,79.338144,8399926,1968724,95,47,12,32
2,122294,15,1,10,245,2017-11-09 04:03:08,0.0,NaN,4,9,...,6,16,4.250000,41.046478,3535169,618358,80,31,12,13
3,73258,9,1,25,145,2017-11-09 04:03:08,0.0,NaN,4,9,...,2,5,4.000000,75.306111,7860226,1441055,82,34,12,21
4,73347,15,1,13,430,2017-11-09 04:03:08,0.0,NaN,4,9,...,12,3,8.166667,41.046478,3535169,188090,83,35,12,22


In [10]:
gc.collect()

20

####  Clicks on app ad before & after

In [11]:
HISTORY_CLICKS = {
    'identical_clicks': ['ip', 'app', 'device', 'os', 'channel'],
    'app_clicks': ['ip', 'app']
}

# Go through different group-by combinations
for fname, fset in HISTORY_CLICKS.items():
    # Clicks in the past
    train_df['prev_'+fname] = train_df.groupby(fset).cumcount().rename('prev_'+fname)
#     # Clicks in the future
#     train_df['future_'+fname] = train_df.iloc[::-1].groupby(fset).cumcount().rename('future_'+fname).iloc[::-1]

# Count cumulative subsequent clicks
train_df.head()

,ip,app,device,os,channel,click_time,is_attributed,click_id,hour,day,...,ip_app_channel_mean_hour,app_AvgViewPerDistinct_ip,app_count_channel,channel_count_app,ip_nunique_channel,ip_nunique_app,ip_day_nunique_hour,ip_app_nunique_os,prev_identical_clicks,prev_app_clicks
0,33924,15,1,19,111,2017-11-09 04:03:08,0.0,NaN,4,9,...,9.166667,41.046478,3535169,187255,95,45,12,34,0,0
1,37383,3,1,13,280,2017-11-09 04:03:08,0.0,NaN,4,9,...,5.215686,79.338144,8399926,1968724,95,47,12,32,0,0
2,122294,15,1,10,245,2017-11-09 04:03:08,0.0,NaN,4,9,...,4.250000,41.046478,3535169,618358,80,31,12,13,0,0
3,73258,9,1,25,145,2017-11-09 04:03:08,0.0,NaN,4,9,...,4.000000,75.306111,7860226,1441055,82,34,12,21,0,0
4,73347,15,1,13,430,2017-11-09 04:03:08,0.0,NaN,4,9,...,8.166667,41.046478,3535169,188090,83,35,12,22,0,0


#### Save FE Process

In [12]:
train_df.to_csv(exp_path+'train_processed.csv', index=False)

In [13]:
train_df.head()

,ip,app,device,os,channel,click_time,is_attributed,click_id,hour,day,...,ip_app_channel_mean_hour,app_AvgViewPerDistinct_ip,app_count_channel,channel_count_app,ip_nunique_channel,ip_nunique_app,ip_day_nunique_hour,ip_app_nunique_os,prev_identical_clicks,prev_app_clicks
0,33924,15,1,19,111,2017-11-09 04:03:08,0.0,NaN,4,9,...,9.166667,41.046478,3535169,187255,95,45,12,34,0,0
1,37383,3,1,13,280,2017-11-09 04:03:08,0.0,NaN,4,9,...,5.215686,79.338144,8399926,1968724,95,47,12,32,0,0
2,122294,15,1,10,245,2017-11-09 04:03:08,0.0,NaN,4,9,...,4.250000,41.046478,3535169,618358,80,31,12,13,0,0
3,73258,9,1,25,145,2017-11-09 04:03:08,0.0,NaN,4,9,...,4.000000,75.306111,7860226,1441055,82,34,12,21,0,0
4,73347,15,1,13,430,2017-11-09 04:03:08,0.0,NaN,4,9,...,8.166667,41.046478,3535169,188090,83,35,12,22,0,0


### Confidence Rates for is_attributed

#### Save FE Process

## Pré-Processing

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58790469 entries, 0 to 58790468
Data columns (total 29 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   ip                             uint32 
 1   app                            uint16 
 2   device                         uint16 
 3   os                             uint16 
 4   channel                        uint16 
 5   click_time                     object 
 6   is_attributed                  float64
 7   click_id                       float64
 8   hour                           uint8  
 9   day                            uint8  
 10  minute                         uint8  
 11  second                         uint8  
 12  ip_app_channel_var_day         float64
 13  ip_app_os_var_hour             float64
 14  ip_day_channel_var_hour        float64
 15  ip_day_hour_count_channel      int64  
 16  ip_app_count_channel           int64  
 17  ip_app_os_count_channel        int64  
 18  

In [15]:
# only train data
display(train_df[:len_train].isna().sum())

# only test data
display(train_df[len_train:].isna().sum())

ip                                      0
app                                     0
device                                  0
os                                      0
channel                                 0
click_time                              0
is_attributed                           0
click_id                         40000000
hour                                    0
day                                     0
minute                                  0
second                                  0
ip_app_channel_var_day            1363065
ip_app_os_var_hour                3173754
ip_day_channel_var_hour            837762
ip_day_hour_count_channel               0
ip_app_count_channel                    0
ip_app_os_count_channel                 0
ip_app_day_hour_count_channel           0
ip_app_channel_mean_hour                0
app_AvgViewPerDistinct_ip               0
app_count_channel                       0
channel_count_app                       0
ip_nunique_channel                

ip                                      0
app                                     0
device                                  0
os                                      0
channel                                 0
click_time                              0
is_attributed                    18790469
click_id                                0
hour                                    0
day                                     0
minute                                  0
second                                  0
ip_app_channel_var_day            1063047
ip_app_os_var_hour                1911723
ip_day_channel_var_hour            851798
ip_day_hour_count_channel               0
ip_app_count_channel                    0
ip_app_os_count_channel                 0
ip_app_day_hour_count_channel           0
ip_app_channel_mean_hour                0
app_AvgViewPerDistinct_ip               0
app_count_channel                       0
channel_count_app                       0
ip_nunique_channel                

In [16]:
para aqui

SyntaxError: invalid syntax (<ipython-input-16-b731f3ce2097>, line 1)

#### Save Pre-Processing Process